In [1]:
import math
import geopandas as gpd
from shapely.geometry import Polygon, Point
from shapely.ops import unary_union
import streamlit as st
import osmnx as ox
import pandas as pd
import ast
import numpy as np

# -----------------------------
# Генерация hex-grid
# -----------------------------
def make_hexagon(center_x, center_y, radius):
    angles_deg = [0, 60, 120, 180, 240, 300]
    pts = [(center_x + radius * math.cos(math.radians(a)), center_y + radius * math.sin(math.radians(a))) for a in angles_deg]
    return Polygon(pts)

def generate_hex_grid(_city_gdf, hex_diameter_km=2.0):
    city = gpd.GeoSeries([_city_gdf.geometry.iloc[0]], crs="EPSG:4326").to_crs(epsg=3857)
    poly = city.iloc[0]

    minx, miny, maxx, maxy = poly.bounds
    radius_m = (hex_diameter_km * 1000.0) / 2.0
    h_step = 1.5 * radius_m
    v_step = math.sqrt(3) * radius_m
    pad = max(radius_m*2, 1000)
    start_x = minx - pad
    end_x = maxx + pad
    start_y = miny - pad
    end_y = maxy + pad

    hexes, centers = [], []
    col = 0
    x = start_x
    while x <= end_x:
        y_offset = v_step / 2.0 if col % 2 == 1 else 0.0
        y = start_y + y_offset
        while y <= end_y:
            hex_poly = make_hexagon(x, y, radius_m)
            inter = hex_poly.intersection(poly)
            if not inter.is_empty:
                hexes.append(inter)
                centers.append((x, y))
            y += v_step
        x += h_step
        col += 1

    hex_gdf = gpd.GeoDataFrame({
        "geometry": hexes,
        "center_x": [c[0] for c in centers],
        "center_y": [c[1] for c in centers],
    }, crs="EPSG:3857")
    hex_gdf["hex_id"] = [f"hex_{i}" for i in range(len(hex_gdf))]
    hex_gdf = hex_gdf.set_index("hex_id").to_crs(epsg=4326)
    centers_gs = gpd.GeoSeries([Point(xy) for xy in zip(hex_gdf["center_x"], hex_gdf["center_y"])], crs="EPSG:3857")
    centers_wgs = centers_gs.to_crs(epsg=4326)
    hex_gdf["center_lon"] = centers_wgs.x.values
    hex_gdf["center_lat"] = centers_wgs.y.values
    hex_gdf_m = hex_gdf.to_crs(epsg=3857)
    hex_gdf["area_m2"] = hex_gdf_m.geometry.area.values
    return hex_gdf

# -----------------------------
# Загружаем границу Москвы (OSM) без Новой Москвы
# -----------------------------
def get_moscow_polygon():
    gdf = ox.geocode_to_gdf("Moscow, Russia").to_crs(epsg=4326)
    geom0 = gdf.geometry.iloc[0]
    if geom0.geom_type == "MultiPolygon":
        largest_poly = max(geom0.geoms, key=lambda p: p.area)
    else:
        largest_poly = geom0
    moscow_gdf = gpd.GeoDataFrame(index=[0], crs="EPSG:4326", geometry=[largest_poly])

    # Троицкий и Новомосковский АО
    troitsky = ox.geocode_to_gdf("Troitsky, Moscow, Russia").to_crs(epsg=4326)
    novomoskovsky = ox.geocode_to_gdf("Novomoskovsky, Moscow, Russia").to_crs(epsg=4326)
    new_moscow_poly = unary_union(list(troitsky.geometry) + list(novomoskovsky.geometry))

    def remove_new_moscow(geom):
        if geom.is_empty:
            return None
        diff = geom.difference(new_moscow_poly)
        if diff.is_empty:
            return None
        return diff

    moscow_gdf["geometry"] = moscow_gdf["geometry"].apply(remove_new_moscow)
    moscow_gdf = moscow_gdf[moscow_gdf["geometry"].notnull()]
    return moscow_gdf

/Users/harddimas24/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def load_parks(json_path="parks.json"):
    df = pd.read_json(json_path, encoding="utf-8")
    park_polygons = []

    for i, geom_str in enumerate(df['geoData']):
        if pd.isna(geom_str):
            continue
        try:
            geom_dict = ast.literal_eval(geom_str)
        except:
            continue
        coords = geom_dict.get('coordinates', None)
        if not coords:
            continue
        try:
            if isinstance(coords[0][0][0], (float, int)):
                park_polygons.append(Polygon(coords[0]))
            else:
                for poly_coords in coords:
                    park_polygons.append(Polygon(poly_coords[0]))
        except:
            continue

    parks_gdf = gpd.GeoDataFrame(geometry=park_polygons, crs="EPSG:4326")
    parks_gdf["geometry"] = parks_gdf.buffer(0)
    print(f"Парков загружено: {len(parks_gdf)}")
    return parks_gdf

# -----------------------------
# Функция расчёта park_score
# -----------------------------
def calc_park_score(hex_poly, parks_gdf):
    hex_area = hex_poly.area
    total_ratio = 0
    for park_poly in parks_gdf.geometry:
        inter = park_poly.intersection(hex_poly)
        if not inter.is_empty:
            total_ratio += inter.area / hex_area

    if total_ratio < 0.05:
        return 0
    elif total_ratio <= 0.20:
        return (total_ratio - 0.05) / 0.15
    elif total_ratio <= 0.40:
        return 1 - (total_ratio - 0.20) / 0.20
    else:
        return 0


def smooth_park_scores(hex_gdf, column="park_score", weight_self=0.7, weight_neighbors=0.3):
    """Сглаживает park_score, добавляя влияние соседних сот."""
    hex_gdf = hex_gdf.copy()

    # Переводим в проекцию в метрах, чтобы точно искать соседей
    gdf_m = hex_gdf.to_crs(epsg=3857)

    # Пространственный индекс для быстрого поиска соседей
    sindex = gdf_m.sindex
    smoothed_scores = []

    for idx, row in gdf_m.iterrows():
        geom = row.geometry
        score = row[column]

        # Поиск кандидатов в радиусе (примерно две ширины соты)
        possible_matches_idx = list(sindex.intersection(geom.buffer(2500).bounds))
        neighbors = gdf_m.iloc[possible_matches_idx]

        # Соседи, которые реально касаются
        neighbors = neighbors[neighbors.intersects(geom)]

        if len(neighbors) > 1:
            neighbor_scores = neighbors[column].values
            avg_neighbor_score = np.mean(neighbor_scores)
            new_score = weight_self * score + weight_neighbors * avg_neighbor_score
        else:
            new_score = score

        smoothed_scores.append(new_score)

    hex_gdf["smoothed_score"] = smoothed_scores
    return hex_gdf

# -----------------------------
# Генерируем сетку и загружаем парки
# -----------------------------
moscow_gdf = get_moscow_polygon()
hex_gdf = generate_hex_grid(moscow_gdf, hex_diameter_km=2.0)
parks_gdf = load_parks("parks.json")

# Переводим в метрическую проекцию для расчёта площадей пересечения
hex_gdf_m = hex_gdf.to_crs(epsg=3857)
parks_gdf_m = parks_gdf.to_crs(epsg=3857)

# -----------------------------
# Считаем park_score
# -----------------------------
scores = []
for i, hex_poly in enumerate(hex_gdf_m.geometry):
    score = calc_park_score(hex_poly, parks_gdf_m)
    scores.append(score)
    if i < 5:
        print(f"Сота {i}: park_score={score:.3f}")

hex_gdf["park_score"] = scores

Парков загружено: 3715
Сота 0: park_score=0.000
Сота 1: park_score=0.136
Сота 2: park_score=0.000
Сота 3: park_score=0.285
Сота 4: park_score=0.000


In [3]:
def boost_neighbors(
    hex_gdf,
    column="park_score",
    influence_radius_m=2000,
    boost_factor=0.07,
    new_col=None,
    min_base_score=0.1
):
    """
    Усиливает значение score у соседних сот рядом с высокими.
    
    Работает с любым столбцом score (park_score, school_score и т.д.).
    - influence_radius_m — радиус влияния (в метрах)
    - boost_factor — доля разницы, добавляемая соседям (0.05–0.15 обычно)
    - new_col — если задано, результат сохраняется в новом столбце
    - min_base_score — минимальный порог, с которого сота начинает влиять на соседей
    """
    # --- Копируем и переводим в метрическую систему ---
    gdf_m = hex_gdf.to_crs(epsg=3857).copy()
    sindex = gdf_m.sindex
    scores = gdf_m[column].fillna(0).astype(float).copy()

    # --- Итерация по сотам ---
    for idx, row in gdf_m.iterrows():
        base_score = scores[idx]
        if base_score < min_base_score:
            continue  # очень слабые соты не распространяют влияние

        geom = row.geometry
        # Ищем кандидатов в радиусе
        possible_idx = list(sindex.intersection(geom.buffer(influence_radius_m).bounds))
        neighbors = gdf_m.iloc[possible_idx]
        neighbors = neighbors[neighbors.intersects(geom.buffer(influence_radius_m))]

        # Повышаем значения у соседей, если они слабее
        for n_idx in neighbors.index:
            if n_idx == idx:
                continue
            diff = base_score - scores[n_idx]
            if diff > 0:
                scores[n_idx] += boost_factor * diff

    # --- Ограничиваем диапазон ---
    boosted = np.clip(scores, 0, 1)

    # --- Обновляем GeoDataFrame ---
    result = gdf_m.copy()
    if new_col:
        result[new_col] = boosted
    else:
        result[column] = boosted

    return result.to_crs(epsg=4326)

In [4]:
hex_gdf = boost_neighbors(hex_gdf,
                          column="park_score",
                          influence_radius_m=2000,
                          boost_factor=0.07)

In [5]:
def load_schools(csv_path="schools.csv"):
    """Загружает школы из CSV с колонками lat, lon → GeoDataFrame."""
    df = pd.read_csv(csv_path)
    df = df[df["lat"].notnull() & df["lon"].notnull()]
    schools_gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["lon"], df["lat"]),
        crs="EPSG:4326"
    )
    print(f"Загружено школ: {len(schools_gdf)}")
    return schools_gdf


def count_schools_per_hex(hex_gdf, schools_gdf, max_good=4, alpha=1.0):
    """
    Считает количество школ внутри каждой соты и добавляет school_score.

    max_good — при каком количестве школ достигается score = 1.0 (например, 4)
    alpha — крутизна кривой роста (0.7–1.2 обычно)
    """
    # --- Переводим в метрическую систему ---
    hex_m = hex_gdf.to_crs(epsg=3857).copy()
    schools_m = schools_gdf.to_crs(epsg=3857)

    # --- Обеспечим наличие hex_id как столбца ---
    if "hex_id" not in hex_m.columns:
        hex_m = hex_m.reset_index()

    # --- Пространственное соединение ---
    joined = gpd.sjoin(schools_m, hex_m, predicate="within")

    # --- Подсчёт количества школ по hex_id ---
    counts = joined.groupby("hex_id").size()

    # --- Добавляем столбец school_amount ---
    hex_m["school_amount"] = hex_m["hex_id"].map(counts).fillna(0).astype(int)

    # --- Расчёт school_score (экспоненциальное насыщение, 4+ школ = 1.0) ---
    c = hex_m["school_amount"].astype(float)
    num = 1 - np.exp(-alpha * c)
    den = 1 - np.exp(-alpha * max_good)
    hex_m["school_score"] = np.clip(num / den, 0, 1)

    # --- Возвращаем в исходную проекцию ---
    return hex_m.to_crs(epsg=4326)

In [6]:
schools_gdf = load_schools("schools.csv")
hex_gdf = count_schools_per_hex(hex_gdf, schools_gdf, max_good=4, alpha=1.0)

Загружено школ: 1554


In [7]:
hex_gdf

,hex_id,geometry,center_x,center_y,center_lon,center_lat,area_m2,park_score,school_amount,school_score
0,hex_0,"MULTIPOLYGON (((37.33072 55.68673, 37.32967 55...",4.154637e+06,7.496304e+06,37.321736,55.686731,3.115363e+04,0.000000,0,0.000000
1,hex_1,"POLYGON ((37.3397 55.64285, 37.34419 55.63846,...",4.156137e+06,7.486777e+06,37.335211,55.638460,1.581461e+06,0.218303,1,0.643914
2,hex_2,"POLYGON ((37.3397 55.65163, 37.34419 55.64724,...",4.156137e+06,7.488509e+06,37.335211,55.647241,1.799884e+06,0.200465,1,0.643914
3,hex_3,"MULTIPOLYGON (((37.34419 55.65602, 37.34268 55...",4.156137e+06,7.490242e+06,37.335211,55.656020,2.534152e+05,0.386949,0,0.000000
4,hex_4,"POLYGON ((37.33072 55.68673, 37.3397 55.68673,...",4.156137e+06,7.495438e+06,37.335211,55.682345,1.641514e+06,0.000000,0,0.000000
...,...,...,...,...,...,...,...,...,...,...
1401,hex_1401,"POLYGON ((37.95056 55.70427, 37.94607 55.70865...",4.225137e+06,7.500634e+06,37.955049,55.708652,1.273320e+06,0.074260,0,0.000000
1402,hex_1402,"POLYGON ((37.95954 55.71303, 37.95056 55.71303...",4.225137e+06,7.502366e+06,37.955049,55.717417,1.843820e+06,0.075278,0,0.000000
1403,hex_1403,"MULTIPOLYGON (((37.95954 55.67796, 37.96345 55...",4.226637e+06,7.494572e+06,37.968523,55.677959,2.282898e+05,0.000000,0,0.000000
1404,hex_1404,"POLYGON ((37.95954 55.68673, 37.96105 55.68821...",4.226637e+06,7.496304e+06,37.968523,55.686731,2.417876e+05,0.015547,0,0.000000


In [8]:
hex_gdf = boost_neighbors(hex_gdf,
                          column="school_score",
                          influence_radius_m=2000,
                          boost_factor=0.005)

In [9]:
hex_gdf['school_score'].unique()

array([0.        , 0.64687318, 0.01456834, ..., 0.01256942, 0.01139381,
       0.00644369])

In [53]:
hex_gdf.to_file("hex_grid_with_score.gpkg", layer="hexes", driver="GPKG")
print("Готово! Файл 'hex_grid_with_score.gpkg' сохранён.")

Готово! Файл 'hex_grid_with_score.gpkg' сохранён.


In [19]:
metro = pd.read_csv('/Users/harddimas24/python/project/metro.csv', sep=None, engine='python', skiprows=1)
metro

,Локальный идентификатор,Наименование,На территории Москвы,Административный округ,Район,Долгота в WGS-84,Широта в WGS-84,Тип вестибюля,Станция метрополитена,Линия,...,Режим работы по нечётным дням,Количество полнофункциональных БПА (все типы билетов),Количество малофункциональных БПА (билеты на 1 и 2 поездки),Общее количество БПА,Ремонт эскалаторов,Статус объекта,global_id,geoData,geodata_center,Unnamed: 21
0,331,"Китай-город, вход-выход 5 в северный вестибюль",да,Центральный административный округ,Басманный район,37.631765,55.757328,подземный,Китай-город,Калужско-Рижская линия,...,открытие в 05:30:00; закрытие в 01:00:00; перв...,NaN,4.0,4.0,nested data,действует,1773539,"{coordinates=[37.63176509, 55.75732811], type=...","{coordinates=[37.63176509, 55.75732811], type=...",NaN
1,327,"Китай-город, вход-выход 4 в северный вестибюль",да,Центральный административный округ,Тверской район,37.630924,55.756733,подземный,Китай-город,Калужско-Рижская линия,...,открытие в 05:30:00; закрытие в 01:00:00; перв...,NaN,4.0,4.0,nested data,действует,1773540,"{coordinates=[37.63092407, 55.75673268], type=...","{coordinates=[37.63092407, 55.75673268], type=...",NaN
2,330,"Китай-город, вход-выход 7 в северный вестибюль",да,Центральный административный округ,Басманный район,37.631862,55.757027,подземный,Китай-город,Калужско-Рижская линия,...,открытие в 05:30:00; закрытие в 01:00:00; перв...,NaN,4.0,4.0,nested data,действует,1773541,"{coordinates=[37.63186197, 55.75702717], type=...","{coordinates=[37.63186197, 55.75702717], type=...",NaN
3,322,"Китай-город, вход-выход 13 в южный вестибюль",да,Центральный административный округ,Тверской район,37.633207,55.752999,подземный,Китай-город,Калужско-Рижская линия,...,открытие в 05:30:00; закрытие в 01:00:00; перв...,NaN,4.0,4.0,nested data,действует,1773542,"{coordinates=[37.63320674, 55.7529987], type=P...","{coordinates=[37.63320674, 55.7529987], type=P...",NaN
4,321,"Китай-город, вход-выход 12 в южный вестибюль",да,Центральный административный округ,Таганский район,37.633566,55.753071,подземный,Китай-город,Калужско-Рижская линия,...,открытие в 05:30:00; закрытие в 01:00:00; перв...,NaN,4.0,4.0,nested data,действует,1773543,"{coordinates=[37.63356616, 55.75307131], type=...","{coordinates=[37.63356616, 55.75307131], type=...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,1505,"Академическая, вход-выход 11 в вестибюль 2",да,Юго-Западный административный округ,Академический район,37.577569,55.686795,подземный,Академическая,Троицкая линия,...,открытие в 05:30:00; закрытие в 01:00:00; перв...,2.0,NaN,2.0,nested data,действует,2801754634,"{coordinates=[37.57756899, 55.68679519], type=...","{coordinates=[37.57756899, 55.68679519], type=...",NaN
1153,1506,"Крымская, вход-выход 1 в вестибюль 2",да,Южный административный округ,Донской район,37.610684,55.689777,наземный отдельностоящий,Крымская,Троицкая линия,...,открытие в 05:30:00; закрытие в 01:00:00; перв...,3.0,NaN,3.0,nested data,действует,2801754647,"{coordinates=[37.61068389, 55.68977709], type=...","{coordinates=[37.61068389, 55.68977709], type=...",NaN
1154,1507,"Крымская, вход-выход 2 в вестибюль 1",да,Южный административный округ,Донской район,37.607165,55.690164,наземный отдельностоящий,Крымская,Троицкая линия,...,открытие в 05:30:00; закрытие в 01:00:00; перв...,2.0,NaN,2.0,nested data,действует,2801754672,"{coordinates=[37.60716498, 55.69016404], type=...","{coordinates=[37.60716498, 55.69016404], type=...",NaN
1155,1508,"ЗИЛ, вход-выход 1 в вестибюль",да,Южный административный округ,Даниловский район,37.643320,55.696151,подземный,ЗИЛ,Троицкая линия,...,открытие в 05:30:00; закрытие в 01:00:00; перв...,7.0,NaN,7.0,nested data,действует,2801754725,"{coordinates=[37.64331963, 55.69615141], type=...","{coordinates=[37.64331963, 55.69615141], type=...",NaN


In [21]:
metro.columns

Index(['Локальный идентификатор', 'Наименование', 'На территории Москвы',
       'Административный округ', 'Район', 'Долгота в WGS-84',
       'Широта в WGS-84', 'Тип вестибюля', 'Станция метрополитена', 'Линия',
       'Статус объекта культурного наследия', 'Режим работы по чётным дням',
       'Режим работы по нечётным дням',
       'Количество полнофункциональных БПА (все типы билетов)',
       'Количество малофункциональных БПА (билеты на 1 и 2 поездки)',
       'Общее количество БПА', 'Ремонт эскалаторов', 'Статус объекта',
       'global_id', 'geoData', 'geodata_center', 'Unnamed: 21'],
      dtype='object')

In [20]:
mcd = pd.read_csv('/Users/harddimas24/python/project/mcd.csv', sep=None, engine='python', skiprows=1)
mcd

,Наименование прохода станции МЦД,Станция МЦД,Тип прохода,На территории Москвы,Административный округ,Район,Широта прохода в WGS-84,Долгота прохода в WGS-84,Ближайшие входы и выходы вестибюлей Московского метрополитена,Статус объекта,global_id,geoData,geodata_center,Unnamed: 13
0,Одинцово (проход 1),nested data,Вход-выход,нет,NaN,NaN,55.672198,37.281430,NaN,действует,1059314872,"{coordinates=[37.28143, 55.672198], type=Point}",NaN,NaN
1,Одинцово (проход 2),nested data,Вход-выход,нет,NaN,NaN,55.671064,37.280726,NaN,действует,1059315973,"{coordinates=[37.280726, 55.671064], type=Point}",NaN,NaN
2,Одинцово (проход 3),nested data,Вход-выход,нет,NaN,NaN,55.673384,37.283735,NaN,действует,1059316135,"{coordinates=[37.283735, 55.673384], type=Point}",NaN,NaN
3,Одинцово (проход 4),nested data,Вход-выход,нет,NaN,NaN,55.672908,37.284388,NaN,действует,1059316279,"{coordinates=[37.284388, 55.672908], type=Point}",NaN,NaN
4,Баковка (проход 1),nested data,Вход-выход,нет,NaN,NaN,55.682354,37.312410,NaN,действует,1059317055,"{coordinates=[37.31241, 55.682354], type=Point}",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,Солнечная (проход 3),nested data,Вход-выход,да,Западный административный округ,район Солнцево,55.656602,37.384801,NaN,действует,2696710139,"{coordinates=[37.384801, 55.656602], type=Point}",NaN,NaN
393,Белорусская (проход 1) (Б),nested data,Вход-выход,да,Центральный административный округ,Тверской район,55.776541,37.581890,NaN,действует,2696710242,"{coordinates=[37.58189, 55.776541], type=Point}",NaN,NaN
394,Белорусская (проход 2) (Б),nested data,Вход-выход,да,Центральный административный округ,Тверской район,55.776874,37.581719,NaN,действует,2696710245,"{coordinates=[37.581719, 55.776874], type=Point}",NaN,NaN
395,Савёловская (проход 5),nested data,Вход-выход,да,Северо-Восточный административный округ,Бутырский район,55.794584,37.588955,NaN,действует,2696710288,"{coordinates=[37.588955, 55.794584], type=Point}",NaN,NaN


In [22]:
mcd.columns

Index(['Наименование прохода станции МЦД', 'Станция МЦД', 'Тип прохода',
       'На территории Москвы', 'Административный округ', 'Район',
       'Широта прохода в WGS-84', 'Долгота прохода в WGS-84',
       'Ближайшие входы и выходы вестибюлей Московского метрополитена',
       'Статус объекта', 'global_id', 'geoData', 'geodata_center',
       'Unnamed: 13'],
      dtype='object')

In [23]:
mcd = mcd.rename(columns={
    'Наименование прохода станции МЦД': 'Наименование',
    'Долгота прохода в WGS-84': 'Долгота в WGS-84',
    'Широта прохода в WGS-84': 'Широта в WGS-84'
})

In [25]:
df_all = pd.concat([metro, mcd], ignore_index=True)

In [26]:
df_all

,Локальный идентификатор,Наименование,На территории Москвы,Административный округ,Район,Долгота в WGS-84,Широта в WGS-84,Тип вестибюля,Станция метрополитена,Линия,...,Ремонт эскалаторов,Статус объекта,global_id,geoData,geodata_center,Unnamed: 21,Станция МЦД,Тип прохода,Ближайшие входы и выходы вестибюлей Московского метрополитена,Unnamed: 13
0,331.0,"Китай-город, вход-выход 5 в северный вестибюль",да,Центральный административный округ,Басманный район,37.631765,55.757328,подземный,Китай-город,Калужско-Рижская линия,...,nested data,действует,1773539,"{coordinates=[37.63176509, 55.75732811], type=...","{coordinates=[37.63176509, 55.75732811], type=...",NaN,NaN,NaN,NaN,NaN
1,327.0,"Китай-город, вход-выход 4 в северный вестибюль",да,Центральный административный округ,Тверской район,37.630924,55.756733,подземный,Китай-город,Калужско-Рижская линия,...,nested data,действует,1773540,"{coordinates=[37.63092407, 55.75673268], type=...","{coordinates=[37.63092407, 55.75673268], type=...",NaN,NaN,NaN,NaN,NaN
2,330.0,"Китай-город, вход-выход 7 в северный вестибюль",да,Центральный административный округ,Басманный район,37.631862,55.757027,подземный,Китай-город,Калужско-Рижская линия,...,nested data,действует,1773541,"{coordinates=[37.63186197, 55.75702717], type=...","{coordinates=[37.63186197, 55.75702717], type=...",NaN,NaN,NaN,NaN,NaN
3,322.0,"Китай-город, вход-выход 13 в южный вестибюль",да,Центральный административный округ,Тверской район,37.633207,55.752999,подземный,Китай-город,Калужско-Рижская линия,...,nested data,действует,1773542,"{coordinates=[37.63320674, 55.7529987], type=P...","{coordinates=[37.63320674, 55.7529987], type=P...",NaN,NaN,NaN,NaN,NaN
4,321.0,"Китай-город, вход-выход 12 в южный вестибюль",да,Центральный административный округ,Таганский район,37.633566,55.753071,подземный,Китай-город,Калужско-Рижская линия,...,nested data,действует,1773543,"{coordinates=[37.63356616, 55.75307131], type=...","{coordinates=[37.63356616, 55.75307131], type=...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1549,NaN,Солнечная (проход 3),да,Западный административный округ,район Солнцево,37.384801,55.656602,NaN,NaN,NaN,...,NaN,действует,2696710139,"{coordinates=[37.384801, 55.656602], type=Point}",NaN,NaN,nested data,Вход-выход,NaN,NaN
1550,NaN,Белорусская (проход 1) (Б),да,Центральный административный округ,Тверской район,37.581890,55.776541,NaN,NaN,NaN,...,NaN,действует,2696710242,"{coordinates=[37.58189, 55.776541], type=Point}",NaN,NaN,nested data,Вход-выход,NaN,NaN
1551,NaN,Белорусская (проход 2) (Б),да,Центральный административный округ,Тверской район,37.581719,55.776874,NaN,NaN,NaN,...,NaN,действует,2696710245,"{coordinates=[37.581719, 55.776874], type=Point}",NaN,NaN,nested data,Вход-выход,NaN,NaN
1552,NaN,Савёловская (проход 5),да,Северо-Восточный административный округ,Бутырский район,37.588955,55.794584,NaN,NaN,NaN,...,NaN,действует,2696710288,"{coordinates=[37.588955, 55.794584], type=Point}",NaN,NaN,nested data,Вход-выход,NaN,NaN


In [28]:
df_all.isnull().sum()

Локальный идентификатор                                           397
Наименование                                                        0
На территории Москвы                                                0
Административный округ                                            164
Район                                                             164
Долгота в WGS-84                                                    0
Широта в WGS-84                                                     0
Тип вестибюля                                                     397
Станция метрополитена                                             397
Линия                                                             397
Статус объекта культурного наследия                               397
Режим работы по чётным дням                                       397
Режим работы по нечётным дням                                     397
Количество полнофункциональных БПА (все типы билетов)             714
Количество малофункц

In [29]:
df_all.to_csv('transport.csv')